# Demo2+ OpenDSS to CYME with post-processing example

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [3]:
#Set the feeder
_feeder='demo_2_plus_latest'

## Read the OpenDSS model into DiTTo

The OpenDSS model for Demo2+ is red exactly as before.

In [4]:
#Build the DiTTo model first...
from ditto.store import Store
from ditto.readers.opendss.read import reader

m=Store()

inputs={'master_file':'../inputs/opendss/{feeder}/master.dss'.format(feeder=_feeder),
        'buscoordinates_file': '../inputs/opendss/{feeder}/buscoords.dss'.format(feeder=_feeder)}

#Instanciate the reader
opendss_reader=reader(**inputs)

#Parse...
opendss_reader.parse(m)

/Users/ngensoll/anaconda2/lib/python2.7/site-packages/fuzzywuzzy-0.15.1-py2.7.egg/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning


'build_opendssdirect' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, '../inputs/opendss/demo_2_plus_latest/master.dss'), {}) 7.32 sec
Build OpenDSSdirect= 7.3166410923
'parse_power_source' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, <ditto.store.Store(elements=0, models=1) object at 0x1514337a10>), {}) 0.01 sec


/Users/ngensoll/GitRepos/OpenDSSDirect.py/opendssdirect/utils.py:98: UserWarning: Pandas cannot be installed. Please see documentation for how to install extra dependencies.
  "Pandas cannot be installed. Please see documentation for how to install extra dependencies."


'parse_nodes' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, <ditto.store.Store(elements=0, models=203815) object at 0x1514337a10>), {}) 355.18 sec
Line class to dataframe= 73.6883239746
rest= 2295.34738493


No handlers could be found for logger "opendssdirect.core"


substation found
substation found
substation found
substation found
substation found
substation found
substation found
substation found
substation found
substation found
'parse_transformers' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, <ditto.store.Store(elements=0, models=647726) object at 0x1514337a10>), {}) 114.65 sec
'parse_loads' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, <ditto.store.Store(elements=0, models=763825) object at 0x1514337a10>), {}) 347.86 sec
'parse_regulators' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, <ditto.store.Store(elements=0, models=763825) object at 0x1514337a10>), {}) 11.60 sec
'parse_capacitors' ((<ditto.readers.opendss.read.reader instance at 0x1514c04c68>, <ditto.store.Store(elements=0, models=764057) object at 0x1514337a10>), {}) 0.51 sec


1

## Modify the DiTTo model

Here begins the interesting stuffs. 
Load the system_structure_modifier module which will handle the modifications to the DiTTo model we just built.

In [5]:
from ditto.modify.system_structure import system_structure_modifier

Instanciate it with the model and the name of the source node ('st_mat' for Demo2+).

In [6]:
modifier=system_structure_modifier(m,'st_mat')

### Center-tap load processing

First, we make sure that everything downstream of center-tap transformers have coherant phases. (Because of the way these are modelled in OpenDSS, we have everything as AB...).

This is done by callling the center_tap_load_preprocessing method:

In [9]:
modifier.center_tap_load_preprocessing()

### Feeder mapping

Then, we wish to have a clean network structure with feeders and sub-transmission networks which is not included in the OpenDSS model.

This can be done by calling the feeder_preprocessing method:

In [7]:
modifier.feeder_preprocessing()

Info: substation ctrafo(cramaee():d2_nssee5_12.47->d2_nssee5_69)d2d3 found downstream of substation ctrafo(cramaee():d2_nst1_69->st_mat)
Info: substation ctrafo(cramaee():d2_nssee3_12.47->d2_nssee3_69)d2d3 found downstream of substation ctrafo(cramaee():d2_nst0_69->st_mat)


The test_feeder_cut method gives some statistic about the feeder created. If some elements are in multiple feeders, the intersections will also be printed (nothing is printed here because feeders are disjoint).

In [8]:
modifier.test_feeder_cut()

Number of feeders defined = 8
Sizes of the feeders = [37528, 38227, 29279, 15999, 10194, 59112, 2958, 49050]


## Write the modified model to CYME

Finally, write the modified model to CYME in the same way as before:

In [10]:
from ditto.writers.cyme.write import writer

In [11]:
cyme_writer=writer(output_path='./',log_path='./log_tt.log')

In [12]:
cyme_writer.write(modifier.model)